In [1]:
import sys
sys.path.append("../src")
from utils import Utils, demUtils
from inpainter import Inpainter
# from landforms import Landforms
# from extents import GetCoordinates
# from processDem import ProcessDem
# from metafunctions import MetaFunctions
import os
import glob
import arcpy
from arcpy.sa import *
arcpy.env.overwriteOutput = True

In [2]:
# Aspect has correct mask
aspect_quarterm = r"C:\Users\ageglio\Desktop\test_dem_products\GHR_2\GHR_processed\quarterm\GHR_aspect_quaterm.tif"
original_coordinate_system = arcpy.Describe(aspect_quarterm).spatialReference
arcpy.env.outputCoordinateSystem = original_coordinate_system

In [3]:
geoform4_q = r"C:\Users\ageglio\Desktop\test_dem_products\GHR_2\GHR_processed\quarterm\GHR_geoform4_quaterm.tif"
geoform5_q = r"C:\Users\ageglio\Desktop\test_dem_products\GHR_2\GHR_processed\quarterm\GHR_geoform5_quaterm.tif"
geoform6_q = r"C:\Users\ageglio\Desktop\test_dem_products\GHR_2\GHR_processed\quarterm\GHR_geoform6_quaterm.tif"
geoform10_q = r"C:\Users\ageglio\Desktop\test_dem_products\GHR_2\GHR_processed\quarterm\GHR_geoform10_quaterm.tif"


In [4]:
geoform4_h = r"C:\Users\ageglio\Desktop\test_dem_products\GHR_2\GHR_processed\halfm\GHR_geoform4_halfm.tif"
geoform5_h = r"C:\Users\ageglio\Desktop\test_dem_products\GHR_2\GHR_processed\halfm\GHR_geoform5_halfm.tif"
geoform6_h = r"C:\Users\ageglio\Desktop\test_dem_products\GHR_2\GHR_processed\halfm\GHR_geoform6_halfm.tif"
geoform10_h = r"C:\Users\ageglio\Desktop\test_dem_products\GHR_2\GHR_processed\halfm\GHR_geoform10_halfm.tif"


## mask out the halfm layers to the missing mask

In [ ]:
# half-m geoform resampled to quarterm
geophorm_h = geoform10_h
geoform_q = geoform10_q
geoform_q_mem = Raster(geoform_q)

# get the binary mask
complete_binary_mask = arcpy.sa.Con(arcpy.sa.IsNull(arcpy.Raster(aspect_quarterm)), 0, 1)
geoform_binary_mask = arcpy.sa.Con(arcpy.sa.IsNull(arcpy.Raster(geoform_q)), 0, 1)
# half-m missing mask
missing_mask_quarterm =  complete_binary_mask - geoform_binary_mask

# We get the values from the half-m (closest available data)
geoform_resampled_halfm = arcpy.ia.Resample(geophorm_h, resampling_type="NearestNeighbor", input_cellsize=0.5, output_cellsize=0.25)
geoform_resampled_halfm_masked = geoform_resampled_halfm * missing_mask_quarterm
geoform_resampled_halfm_valid = SetNull(geoform_resampled_halfm_masked == 0, geoform_resampled_halfm_masked)
geoform_mem = SetNull(geoform_q_mem == 0, geoform_q_mem)

mosaic_raster = arcpy.management.MosaicToNewRaster(
    input_rasters=[geoform_mem, geoform_resampled_halfm_valid],
    output_location=r"memory",  # Use memory workspace for temporary storage
    raster_dataset_name_with_extension="mosaic_result",
    coordinate_system_for_the_raster=original_coordinate_system,  
    pixel_type="", # Optional: match input rasters
    cellsize=0.25, # Optional: match input rasters
    number_of_bands=1,
    mosaic_method="LAST",
    mosaic_colormap_mode="FIRST"
)

name = Utils.sanitize_path_to_name(geoform_q)
cleaned_raster = SetNull(mosaic_raster, mosaic_raster, "VALUE = 0")
cleaned_raster.save(r"C:\Users\ageglio\Desktop\test_dem_products" + os.sep + f"{name}_cleaned.tif")